In [1]:
import pymysql
import csv
import pandas as pd

In [2]:
#可選擇從MSQL或CSV拉資料

In [19]:
sql = "select class_nm, quest_nm_adj, quest_kw, quest_ner from textdb.fact_cls_qa_kw where quest_kw is not null" #SQL指令拉資料

In [20]:
db = pymysql.connect(host='172.16.56.101', port=31996, user='root', passwd='password', db='textdb' )
cursor = db.cursor()
cursor.execute(sql)
kw_results = cursor.fetchall() #從MYSQL拉出資料

In [21]:
csv_data = pd.read_csv('data/fact_cls_qa_kw.csv', encoding='big5') #從CSV拉資料

In [8]:
kw_results = [] #整理CSV資料
for i in range(len(csv_data)):
    kw_results.append([
    list(csv_data['class_nm'])[i],
    list(csv_data['quest_nm_adj'])[i],
    list(csv_data['quest_kw'])[i],
    list(csv_data['quest_ner'])[i]]) 

In [22]:
kw_results[:100] #資料展示前5筆

(('合約查詢', '我現在的資費方案是?', '方案,資費', 'PROJ,PROJ'),
 ('合約查詢', '我的方案是啥', '方案', 'PROJ'),
 ('合約查詢', '請告訴我當初辦理的專案', '專案', 'PROJ'),
 ('合約查詢', '目前資費是什麼', '資費', 'PROJ'),
 ('合約查詢', '我忘了當初申辦的資費了', '資費,申辦', 'PROJ,BID'),
 ('合約查詢', '我想要解約NP去別家', '解約', 'CANCEL'),
 ('合約查詢', '怎麼辦理解約', '解約', 'CANCEL'),
 ('合約查詢', '不想用這個門號了，我要解約', '解約,門號', 'CANCEL,MDN'),
 ('合約查詢', '收訊太爛了，我不想用了啦，立刻給我解約', '解約,收訊', 'CANCEL,SIGNAL'),
 ('合約查詢', '離譜的訊號品質，給我解了吧', '訊號', 'SIGNAL'),
 ('合約查詢', '請問幫我查一下目前門號綁哪個約', '門號', 'MDN'),
 ('合約查詢', '我生活受到影響了，請幫我解約', '解約', 'CANCEL'),
 ('合約查詢', '我的約快到期了吧，查一下', '到期', 'INTERVAL'),
 ('合約查詢', '我還有多久到期', '到期', 'INTERVAL'),
 ('合約查詢', '我要NP啦，違約金要付多少，快講', '違約金', 'BRKM'),
 ('合約查詢', '我的促案是綁什麼資費', '促案,資費', 'PROJ,PROJ'),
 ('合約查詢',
  '我是辦網路吃到飽，最近網路常常無法連線，我現在還有多久到期',
  '到期,連線,網路',
  'INTERVAL,NETWORK,NETWORK'),
 ('合約查詢', '請問我如果解約要賠多少錢?', '解約,賠多少', 'CANCEL,CLAIM'),
 ('合約查詢', '請問目前解約金多少？', '解約金', 'BRKM'),
 ('合約查詢', '想請問提前解約，違約金大概多少呢', '違約金,解約', 'BRKM,CANCEL'),
 ('合約查詢', '所以我可以隨時解約吧?', '解約', 'CANCEL'),
 ('合約查詢', '了解，

In [23]:
import numpy
class_list_orgn = []
for raw in kw_results:
    rsl = raw[3].split(',')
    for rs in rsl:
        class_list_orgn.append(rs)
unique, counts = numpy.unique(class_list_orgn, return_counts=True)
dict(zip(unique, counts)) #統計各個NER個數

{'AMT': 262,
 'BID': 843,
 'BILLCYCLE': 55,
 'BILLMK': 4082,
 'BRANCH': 519,
 'BRKM': 729,
 'CANCEL': 821,
 'CLAIM': 10,
 'DEVICE': 249,
 'ENV': 106,
 'INTERVAL': 655,
 'KPI': 74,
 'MAGIC': 42,
 'MDN': 929,
 'NETWORK': 529,
 'PER': 242,
 'PROJ': 3527,
 'SIGNAL': 305,
 'STATUS': 129}

In [24]:
import numpy
class_list_orgn = []
for raw in kw_results:
    rsl = raw[0].split(',')
    for rs in rsl:
        class_list_orgn.append(rs)
unique, counts = numpy.unique(class_list_orgn, return_counts=True)
dict(zip(unique, counts)) #統計各class筆數

{'合約查詢': 4417, '帳務查詢': 3636, '魔速方塊1.0': 18, '魔速方塊2.0': 30}

In [25]:
class2eng = {'合約查詢': 'planqu', '帳務查詢': 'billqu', '魔速方塊1.0': 'mb1', '魔速方塊2.0': 'mb2'} #把CLASS名稱轉成英文名

In [26]:
#把資料轉成RASA訓練資料格式
"""
範例:
{
    "entities": [
        {
            "end": 4,
            "entity": "BILLCYCLE",
            "start": 0,
            "value": "帳單周期"
        }
    ],
    "intent": "billqu",
    "text": "帳單周期看得到嗎?"
}
"""
dil = []

dataall = {"rasa_nlu_data": {"common_examples":[]}}

for testtext in kw_results:

    data = {}
    data["entities"] = []
    data["intent"] = class2eng[testtext[0]]

    try:

        if len(testtext[3].split(',')) < 2:
            nerl = testtext[3]
            for kwl in testtext[2].split(','):

                entities = {"start": testtext[1].index(kwl), 
                             "end": testtext[1].index(kwl) + len(kwl), 
                             "value": kwl, 
                             "entity":testtext[3]}

                data["entities"].append(entities)
        else:
#                     kwl = testtext[2].split(',')
            nernm = 0
            nerl = testtext[3].split(',')
            for kwl in testtext[2].split(','):


                entities = {"start": testtext[1].index(kwl), 
                             "end": testtext[1].index(kwl) + len(kwl), 
                             "value": kwl, 
                             "entity":nerl[nernm]}

                data["entities"].append(entities)                   
                nernm += 1

    except (ValueError, IndexError):    
        continue

    data["text"] = testtext[1]
    print(data)

    dataall["rasa_nlu_data"]["common_examples"].append(data)

{'entities': [{'start': 6, 'end': 8, 'value': '方案', 'entity': 'PROJ'}, {'start': 4, 'end': 6, 'value': '資費', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '我現在的資費方案是?'}
{'entities': [{'start': 2, 'end': 4, 'value': '方案', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '我的方案是啥'}
{'entities': [{'start': 9, 'end': 11, 'value': '專案', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '請告訴我當初辦理的專案'}
{'entities': [{'start': 2, 'end': 4, 'value': '資費', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '目前資費是什麼'}
{'entities': [{'start': 8, 'end': 10, 'value': '資費', 'entity': 'PROJ'}, {'start': 5, 'end': 7, 'value': '申辦', 'entity': 'BID'}], 'intent': 'planqu', 'text': '我忘了當初申辦的資費了'}
{'entities': [{'start': 3, 'end': 5, 'value': '解約', 'entity': 'CANCEL'}], 'intent': 'planqu', 'text': '我想要解約NP去別家'}
{'entities': [{'start': 4, 'end': 6, 'value': '解約', 'entity': 'CANCEL'}], 'intent': 'planqu', 'text': '怎麼辦理解約'}
{'entities': [{'start': 11, 'end': 13, 'value': '解約', 'entity': 'CANCEL'}, {'start': 5, 'end

{'entities': [{'start': 9, 'end': 11, 'value': '方案', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '那請問現在299的方案可以續嗎？或是針對續約是否有其他優惠呢？謝謝'}
{'entities': [{'start': 6, 'end': 9, 'value': '違約金', 'entity': 'BRKM'}, {'start': 2, 'end': 4, 'value': '解約', 'entity': 'CANCEL'}], 'intent': 'planqu', 'text': '如果解約的話違約金是多少呢?'}
{'entities': [{'start': 9, 'end': 11, 'value': '方案', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '88活動的$288方案'}
{'entities': [{'start': 3, 'end': 5, 'value': '到期', 'entity': 'INTERVAL'}, {'start': 6, 'end': 8, 'value': '速度', 'entity': 'KPI'}], 'intent': 'planqu', 'text': '12期到期後速度和流量?'}
{'entities': [{'start': 6, 'end': 8, 'value': '專案', 'entity': 'PROJ'}, {'start': 13, 'end': 15, 'value': '到期', 'entity': 'INTERVAL'}], 'intent': 'planqu', 'text': '父親節138專案如果一年後到期續用一樣優惠上網加語音？'}
{'entities': [{'start': 17, 'end': 19, 'value': '專案', 'entity': 'PROJ'}, {'start': 8, 'end': 10, 'value': '申請', 'entity': 'BID'}], 'intent': 'planqu', 'text': '本人8/6已成功申請到一門八八節的專案(138)訂單編號：190806087

{'entities': [{'start': 13, 'end': 15, 'value': '合約', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '對我來說很困擾，公司對帳與合約不符，請我們下層查詢'}
{'entities': [{'start': 13, 'end': 15, 'value': '合約', 'entity': 'PROJ'}, {'start': 15, 'end': 17, 'value': '到期', 'entity': 'INTERVAL'}, {'start': 9, 'end': 11, 'value': '門號', 'entity': 'MDN'}], 'intent': 'planqu', 'text': '請問我有其他電信的門號已經合約到期，想要蓋號該怎麼做？'}
{'entities': [{'start': 15, 'end': 19, 'value': 'WIFI', 'entity': 'SIGNAL'}, {'start': 24, 'end': 26, 'value': '方案', 'entity': 'PROJ'}, {'start': 19, 'end': 21, 'value': '網路', 'entity': 'NETWORK'}], 'intent': 'planqu', 'text': '我想詢問有關只有SIM卡FORWIFI網路使用的方案'}
{'entities': [{'start': 15, 'end': 17, 'value': '方案', 'entity': 'PROJ'}, {'start': 13, 'end': 15, 'value': '門號', 'entity': 'MDN'}, {'start': 2, 'end': 4, 'value': '門市', 'entity': 'BRANCH'}, {'start': 0, 'end': 2, 'value': '網路', 'entity': 'NETWORK'}], 'intent': 'planqu', 'text': '網路門市不綁約上網吃到飽單門號方案'}
{'entities': [{'start': 6, 'end': 8, 'value': '方案', 'entity

{'entities': [{'start': 5, 'end': 7, 'value': '合約', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '因為我實際合約是到10月'}
{'entities': [{'start': 14, 'end': 16, 'value': '合約', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '吃到飽到10月左右嗎…跟之前合約一樣…'}
{'entities': [{'start': 12, 'end': 14, 'value': '方案', 'entity': 'PROJ'}, {'start': 2, 'end': 4, 'value': '網路', 'entity': 'NETWORK'}], 'intent': 'planqu', 'text': '請問網路299攜碼吃到飽方案截止日到？'}
{'entities': [{'start': 9, 'end': 11, 'value': '方案', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '目前0月租可轉換此方案嗎？'}
{'entities': [{'start': 5, 'end': 7, 'value': '方案', 'entity': 'PROJ'}, {'start': 18, 'end': 20, 'value': '申請', 'entity': 'BID'}], 'intent': 'planqu', 'text': '11不綁約方案我有取得活動序號，進入申請過程網頁當掉再重開，就說序號以經用掉不給進入申請'}
{'entities': [{'start': 11, 'end': 14, 'value': '解約金', 'entity': 'BRKM'}, {'start': 6, 'end': 8, 'value': '解約', 'entity': 'CANCEL'}], 'intent': 'planqu', 'text': '7月10日如解約有多少解約金'}
{'entities': [{'start': 14, 'end': 16, 'value': '方案', 'entity': 'PROJ'

{'entities': [{'start': 5, 'end': 7, 'value': '方案', 'entity': 'PROJ'}, {'start': 13, 'end': 15, 'value': '客戶', 'entity': 'PER'}], 'intent': 'planqu', 'text': '你們優惠的方案好像都是給新客戶及攜碼的才有'}
{'entities': [{'start': 18, 'end': 20, 'value': '方案', 'entity': 'PROJ'}, {'start': 14, 'end': 16, 'value': '門號', 'entity': 'MDN'}, {'start': 5, 'end': 7, 'value': '門市', 'entity': 'BRANCH'}, {'start': 3, 'end': 5, 'value': '網路', 'entity': 'NETWORK'}], 'intent': 'planqu', 'text': '您是說網路門市吃到飽自由選單門號續約方案12期嗎'}
{'entities': [{'start': 0, 'end': 2, 'value': '專案', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '專案補貼款\t1,200'}
{'entities': [{'start': 2, 'end': 4, 'value': '資費', 'entity': 'PROJ'}], 'intent': 'planqu', 'text': '請問資費內容上面的1200是要付款的嗎'}
{'entities': [{'start': 11, 'end': 13, 'value': '解約', 'entity': 'CANCEL'}], 'intent': 'planqu', 'text': '所以1200是若有提前解約才要付款嗎'}
{'entities': [{'start': 16, 'end': 19, 'value': '違約金', 'entity': 'BRKM'}, {'start': 11, 'end': 13, 'value': '解約', 'entity': 'CANCEL'}], 'intent':

{'entities': [{'start': 8, 'end': 10, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '82/03/27帳單地址一樣'}
{'entities': [{'start': 10, 'end': 12, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '我想詢問一下我八月分帳單多一筆233的金額那是什麼？0906551815'}
{'entities': [{'start': 5, 'end': 7, 'value': '帳單', 'entity': 'BILLMK'}, {'start': 3, 'end': 5, 'value': '手機', 'entity': 'DEVICE'}], 'intent': 'billqu', 'text': '您好我手機帳單有代付款項功能'}
{'entities': [{'start': 3, 'end': 5, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '你們有帳單細項但只有編號沒有名稱'}
{'entities': [{'start': 9, 'end': 11, 'value': '繳款', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '800115信用卡繳款'}
{'entities': [{'start': 4, 'end': 8, 'value': '帳單週期', 'entity': 'BILLCYCLE'}], 'intent': 'billqu', 'text': '我的9月帳單週期還沒到，~19日，今天就收到0元付款的簡訊？？'}
{'entities': [{'start': 0, 'end': 2, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '帳單地址為台南市新化地區中興路665巷11弄9號'}
{'entities': [{'start': 16, 'end': 18, 'valu

{'entities': [{'start': 21, 'end': 23, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '因為改成這樣，所以我才沒有收到七月份的電子帳單嗎?'}
{'entities': [{'start': 12, 'end': 14, 'value': '帳單', 'entity': 'BILLMK'}, {'start': 33, 'end': 35, 'value': '繳款', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '但是通常不是應該會發電子帳單，註記因未達300元，所以合併下期帳單繳款嗎?'}
{'entities': [{'start': 3, 'end': 5, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '那合併帳單的話，只會列最新一期的明細，這樣也不太對吧'}
{'entities': [{'start': 7, 'end': 9, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '你幫我改回每期帳單分開寄送好了'}
{'entities': [{'start': 13, 'end': 15, 'value': '繳費', 'entity': 'BILLMK'}, {'start': 16, 'end': 18, 'value': '用戶', 'entity': 'PER'}, {'start': 9, 'end': 11, 'value': '適合', 'entity': 'STATUS'}], 'intent': 'billqu', 'text': '你們的這個方式比較適合現金繳費的用戶，已經信用卡扣繳了，改這樣也沒有所謂方便不方便'}
{'entities': [{'start': 14, 'end': 16, 'value': '繳費', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '那如果都沒有使用但還是有持續繳費會有什麼多收維持金之類的嗎?'}
{'e

{'entities': [{'start': 3, 'end': 5, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '住家是帳單地址、工廠是中壢區合圳北路二段505號靠近蘆竹區'}
{'entities': [{'start': 8, 'end': 10, 'value': '門號', 'entity': 'MDN'}, {'start': 25, 'end': 27, 'value': '繳費', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '我名下還有另一個門號0961227517,已是自動繳費'}
{'entities': [{'start': 5, 'end': 7, 'value': '門號', 'entity': 'MDN'}, {'start': 18, 'end': 20, 'value': '繳費', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '我想把這個門號0906712308的繳費方式跟合併到上面那隻'}
{'entities': [{'start': 6, 'end': 8, 'value': '繳費', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '我在7-11繳費，有入帳了嗎？'}
{'entities': [{'start': 15, 'end': 17, 'value': '繳款', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '您好請幫我先復話，我明天才能去繳款'}
{'entities': [{'start': 13, 'end': 15, 'value': '帳單', 'entity': 'BILLMK'}], 'intent': 'billqu', 'text': '沒辦理自動扣款，是辦理簡訊帳單'}
{'entities': [{'start': 19, 'end': 21, 'value': '門號', 'entity': 'MDN'}, {'start': 21, 'end': 23, 'value': '帳單'

In [ ]:
import json #轉乘JSON格式展示
print (json.dumps(dataall, sort_keys=True, indent=4, separators=(',', ': ')))

In [ ]:
with open('./RASA_model/fact_cls_qa_kw.json', 'w') as f: #儲存成JSON格式
    json.dump(dataall, f)

In [ ]:
#RASA訓練
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer

training_data = load_data('./RASA_model/fact_cls_qa_kw.json') #RASA訓練資料位置(JSON)
trainer = Trainer(config.load("./nlu_config.yml")) #RASA訓練設定
trainer.train(training_data)
model_directory = trainer.persist('RASA_model/', fixed_model_name="fact_cls_qa_kw") #model儲存位置

In [ ]:
from rasa_nlu.model import Interpreter #讀取RASA model測試
interpreter = Interpreter.load("./RASA_model/fact_cls_qa_kw/")

In [ ]:
text = '解約金是多少' #RASA model 句子測試
result = interpreter.parse(text)
result